In [17]:
import scipy
import numpy as np
from sklearn.metrics import pairwise_distances
import networkx as nx
import pandas as pd
from ripser import Rips
import matplotlib.pyplot as plt
from sklearn import datasets
# from convhull import convhull

In [13]:
# iris = datasets.load_iris()
# X = iris.data  # we only take the first two features.
# y = iris.target
# POINTS_ARRAY = X

In [38]:
# np.random.seed(10)
# POINTS_ARRAY = np.random.randint(1, 15, size = (6,3))
# # POINTS_ARRAY = np.random.uniform(size = (100,3))
# POINTS_ARRAY

POINTS_ARRAY = pd.read_csv('data/iris.csv', header = None)
POINTS_ARRAY
POINTS_ARRAY = np.array(POINTS_ARRAY)
# POINTS_ARRAY.shape
POINTS_ARRAY

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [ ]:
# xdf = pd.DataFrame(POINTS_ARRAY, columns = ['x1', 'x2', 'x3'])
# xdf.to_csv('3d-data-6points-seed-10.csv', index = False)

In [4]:
distance_matrix = pairwise_distances(POINTS_ARRAY).round(2)
distance_matrix

array([[ 0.  , 16.55, 10.3 ,  9.49,  7.14, 10.3 ],
       [16.55,  0.  , 14.56, 15.62, 12.37,  9.49],
       [10.3 , 14.56,  0.  , 17.44,  4.58, 13.04],
       [ 9.49, 15.62, 17.44,  0.  , 13.75,  7.62],
       [ 7.14, 12.37,  4.58, 13.75,  0.  , 10.25],
       [10.3 ,  9.49, 13.04,  7.62, 10.25,  0.  ]])

In [19]:
mst = scipy.sparse.csgraph.minimum_spanning_tree(distance_matrix)
print(mst)

  (0, 4)	7.14
  (1, 5)	9.49
  (2, 4)	4.58
  (3, 0)	9.49
  (3, 5)	7.62


In [ ]:
G = nx.from_numpy_matrix(distance_matrix, create_using=nx.DiGraph)
layout = nx.spring_layout(G)
nx.draw(G, layout, with_labels = True)
labels = nx.get_edge_attributes(G, "weight")
_ = nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=labels)

In [ ]:
G = nx.from_numpy_matrix(mst.toarray(), create_using=nx.DiGraph)
layout = nx.spring_layout(G)
nx.draw(G, layout, with_labels = True)
labels = nx.get_edge_attributes(G, "weight")
_ = nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=labels)

In [20]:
mst_cords = mst.tocoo()
mst_df = pd.DataFrame({'src':mst_cords.row, 'dst':mst_cords.col, 'dist':mst_cords.data})
mst_df = mst_df.sort_values(by='dist').reset_index(drop = True)
# mst_df.iloc[4,0] = 0
# mst_df.iloc[4,1] = 3
mst_df

,src,dst,dist
0,2,4,4.58
1,0,4,7.14
2,3,5,7.62
3,1,5,9.49
4,3,0,9.49


In [21]:
class Component:
    def __init__(self, vertices, polygon):
        self.vertices = vertices
        self.polygon = polygon

In [22]:
class NaiveDisjoinSet:
    def __init__(self, n_points):
        self.universal = [Component({i}, [(0,0), (0,0)]) for i in range(n_points)]
    def Find(self, a):
        for ele in self.universal:
            if a in ele.vertices:
                return ele
        raise Exception("Element Not Found")
    def Union(self, a, b, merged_hull):
        s1 = self.Find(a)
        s2 = self.Find(b)
        if s1.vertices == s2.vertices:
            raise Exception("Elements belong to same set")
        s3 = s1.vertices.union(s2.vertices)
        nc = Component(s3, merged_hull)
        self.universal.remove(s1)
        self.universal.remove(s2)
        self.universal.append(nc)
        
        
        
# nd = NaiveDisjoinSet(5)
# print(nd.universal[0].vertices)
# nd.Find(4).vertices
# nd.Union(1, 2, [(0,0), (0,1), (0,0)])
# nd.Find(2)
# nd.Union(3, 1)
# nd.Find(1)
# nd.Find(2)
# nd.Find(4)
# nd.Find(2)

In [23]:
# np.Inf

In [24]:
def find_angle(p1, p2, trans):
    print("*"*10)
    x = p2[0] - p1[0]
    y = p2[1] - p1[1]
    n = np.sqrt(x*x + y*y)
    print(f"x: {x}  y: {y} mag: {n}")
    x = x/n
    y = y/n
    print(f"DIV x: {x}  y: {y}")
    trans['cos'] = x
    trans['sin'] = np.sqrt(1 - x*x)
    
    if y >= 0:
        trans['sin'] = -trans['sin']
        print(f"FIND ANGLE NEG trans: (tx,ty):({trans['tx'],trans['ty']})  (sin,cos):({trans['sin'],trans['cos']})")
    print(f"FIND ANGLE trans: (tx,ty):({trans['tx'],trans['ty']})  (sin,cos):({trans['sin'],trans['cos']})")
    print("*"*10)
    return trans
    
    
    

In [25]:
def alignHull(hull, p1, top):
    v= -1
    d2 = np.Inf
#     print(f"Before iteration v: {v} d2: {d2}")
    for i, p2 in enumerate(hull):
        d = np.linalg.norm(np.array(p1) - np.array(p2))
        if d < d2:
            v = i
            d2 = d
#         print(f"Iteration {i} v: {v} d2: {d2}")
    print(f"Align Hull After Iteration v: {v} d: {d2}")
    if top:
        v1 = hull[v]
        v2 = hull[v+1]
    else:
        if v == 0:
            v = -1
        v1 = hull[v]
        v2 = hull[v-1]
        
        
    print(f"Align Hull v1: {v1} v2:{v2}")
    trans = {}
    trans['tx'] = -hull[v][0]
    trans['ty'] = -hull[v][1]
    print(f"trans: {trans}")
    if len(hull) > 2:
        trans = find_angle(v1, v2, trans)
        print(f"COMPUTED trans: (tx,ty):({trans['tx'],trans['ty']})  (sin,cos):({trans['sin'],trans['cos']})")
    else:
        trans['sin'] = 0
        trans['cos'] = 1
        print(f"ASSIGNED trans: (tx,ty):({trans['tx'],trans['ty']})  (sin,cos):({trans['sin'],trans['cos']})")
    return trans
        
        

In [26]:
def transform(p, t, yoff):
    x = p[0] + t['tx']
    y = p[1] + t['ty']
    xx = x*t['cos'] - y*t['sin']
    yy = x*t['sin'] + y*t['cos']
    yy += yoff
    return (xx, yy)

In [27]:
def transform_component(c, t, yoff):
    global verts
    for v in c.vertices:
        verts[v] = transform(verts[v], t, yoff)
        
        
        

In [28]:
# def dummy_convex(itr):
#     chuls = [
#         [(0, 0), (0, 4.58258), (0, 0)],
#         [(0, 0), (0, 7.14143), (4.58258, 7.14143), (0, 0)],
#         [(0, 0), (0, 7.61577), (0, 0)],
#         [(0, 0), (0, 9.48683), (7.61577, 9.48683), (0, 0)],
#         [(0, 0), (0, 9.48683), (7.61577, 18.9737), (7.61577, 9.48683), (7.14143, -4.58258), (0, 0)]
#     ]
#     return chuls[itr]

In [29]:
def compute_convex_hull(points):
    global verts
    convex_hull = []
    if points.shape[0] == 1:
        convex_hull.append(tuple(points[0]))
        convex_hull.append(tuple(points[0]))
    elif points.shape[0] ==2:
        convex_hull.append(tuple(points[0]))
        convex_hull.append(tuple(points[1]))
        convex_hull.append(tuple(points[0]))
    else:
        
        n_hull= list(scipy.spatial.ConvexHull(points).vertices)
        n_hull.append(n_hull[0])
        convex_hull = [tuple(points[i]) for i in n_hull]
    return convex_hull
        

In [32]:
def mergeComponents(c1_o,c2_o,v1,v2,length):
    print('$'*50)
    global verts
    c1 = c1_o.vertices
    c2 = c2_o.vertices
    merged = set(list(c1)+list(c2))
    
    print(f'merged: {merged}')
    if length > 0:
        t1 = alignHull(c1_o.polygon, verts[v1], True)
        print("t1: ", t1)
        transform_component(c1_o, t1, 0)
        t2 = alignHull(c2_o.polygon, verts[v2], False)
        print("t2: ", t2)
        transform_component(c2_o, t2, length)
        points = np.array([list(verts[v]) for v in merged])
        print("points: ", points)
        n_hull= compute_convex_hull(points)
#         n_hull= dummy_convex(itttttt)
        print("n_hull: ",n_hull)
    else:
        raise Exception("Same co-ordinates")
    print('$'*50)
    return n_hull
            
    

In [33]:
nd = NaiveDisjoinSet(len(POINTS_ARRAY))

verts = {}
for i in range(len(POINTS_ARRAY)):
    verts[i] = (0,0)
    
for i in range(len(mst_df)):
    print(f"Iteration: {i}")
    p1 = mst_df["src"][i]
    p2 = mst_df["dst"][i]
    
    print(f"p1: {p1}  p2: {p2}")

    c1 = nd.Find(p1)
    c2 = nd.Find(p2)
    print(f"c1: {c1.vertices}  c2: {c2.vertices}")
    if c1.vertices == c2.vertices :
        print("Error")
    hull = mergeComponents(c1,c2,p1,p2,mst_df["dist"][i])
    nd.Union(p1, p2, hull)
    print('-'*100)
    
    # Plotting
    

print(verts)

Iteration: 0
p1: 2  p2: 4
c1: {2}  c2: {4}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
merged: {2, 4}
Align Hull After Iteration v: 0 d: 0.0
Align Hull v1: (0, 0) v2:(0, 0)
trans: {'tx': 0, 'ty': 0}
ASSIGNED trans: (tx,ty):((0, 0))  (sin,cos):((0, 1))
t1:  {'tx': 0, 'ty': 0, 'sin': 0, 'cos': 1}
Align Hull After Iteration v: 0 d: 0.0
Align Hull v1: (0, 0) v2:(0, 0)
trans: {'tx': 0, 'ty': 0}
ASSIGNED trans: (tx,ty):((0, 0))  (sin,cos):((0, 1))
t2:  {'tx': 0, 'ty': 0, 'sin': 0, 'cos': 1}
points:  [[0.   0.  ]
 [0.   4.58]]
n_hull:  [(0.0, 0.0), (0.0, 4.58), (0.0, 0.0)]
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
----------------------------------------------------------------------------------------------------
Iteration: 1
p1: 0  p2: 4
c1: {0}  c2: {2, 4}
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
merged: {0, 2, 4}
Align Hull After Iteration v: 0 d: 0.0
Align Hull v1: (0, 0) v2:(0, 0)
trans: {'tx': 0, 'ty': 0}
ASSIGNED trans: (tx,ty):((0, 0))  (sin,cos):((0, 1))
t1:

In [ ]:
verts

In [ ]:
POINTS_ARRAY

In [ ]:
tpoints = np.array([list(e) for e in verts.values()])
tpoints

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
ot_df = pd.DataFrame(tpoints)
# ot_df.to_csv('outpy.csv', header = False, index = False)
ot_df['y'] = y
ot_df

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns
sns.scatterplot(data = ot_df, x=0, y= 1, hue = 'y')


In [ ]:
rips = Rips()
diagrams = rips.fit_transform(POINTS_ARRAY)
rips.plot(diagrams)


In [ ]:
rips = Rips()
diagrams = rips.fit_transform(tpoints)
rips.plot(diagrams)


In [ ]:
x_coordinates = [6.720628116858413, 0.0, 12.001925653366255, 0.14961699462148648, 6.580329943064301, 0.0]
y_coordinates = [-8.630437747638283, 0.0, -5.016733973984863, -1.0429676971940935, 1.243519480513788, 9.486832980505138]

In [ ]:
cpoints = np.array([[i, j] for i,j in zip(x_coordinates, y_coordinates)])
cpoints

In [ ]:
rips = Rips()
diagrams = rips.fit_transform(cpoints)
rips.plot(diagrams)


In [ ]:
c1 in nd.universal

In [ ]:
fig, ax = plt.subplots(5, 2)

In [ ]:
verts

In [ ]:
def get_plot_points(d):
    tpoints = np.array([list(e) for e in d.values()])
    return tpoints[:,0],tpoints[:,1]
    

In [ ]:
get_plot_points(verts)

In [ ]:
nd = NaiveDisjoinSet(len(POINTS_ARRAY))

verts = {}
for i in range(len(POINTS_ARRAY)):
    verts[i] = (0,0)
    
fig, ax = plt.subplots(len(mst_df), 2, sharex = True, sharey = True, figsize = (12,24))
    
for i in range(len(mst_df)):
    x_points, y_points = get_plot_points(verts)
    ax[i][0].scatter(x_points, y_points)
    for j in range(len(x_points)):
        ax[i][0].text(x_points[j], y_points[j], j)
        
    p1 = mst_df["src"][i]
    p2 = mst_df["dst"][i]

    c1 = nd.Find(p1)
    c2 = nd.Find(p2)
    print(f"c1: {c1.vertices}  c2: {c2.vertices}")
    if c1.vertices == c2.vertices :
        print("Error")
    hull = mergeComponents(c1,c2,p1,p2,mst_df["dist"][i])
    print(hull)
    nd.Union(p1, p2, hull)
    print('-'*10)
    
    # Plotting
    x_points, y_points = get_plot_points(verts)
    ax[i][1].scatter(x_points, y_points)
    for j in range(len(x_points)):
        ax[i][1].text(x_points[j], y_points[j], j)
    



In [ ]:
plt.show()

In [5]:
a = np.array([[0., 0.],
 [4.58, 7.14],
 [0. ,7.14]])
a.shape

(3, 2)

In [9]:
all_vars = ['Qa','Qc','Qi','QJ','Qt','Qu','Qw','Qbb','Qs','Qv','Qx','Qz','Qbk:n','Qbk:0Bk:0','QbB','QBk:n','QRn','Qg','QGn','QVn']

In [11]:
for var in all_vars:
    try:
        hobj = scipy.spatial.ConvexHull(a,qhull_options = var)
        print(hobj.vertices)
    except:
        print(var)

[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
Qu
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
Qz
Qbk:n
Qbk:0Bk:0
[0 1 2]
QBk:n
QRn
Qg
QGn
QVn


In [7]:
hobj.vertices

array([0, 1, 2], dtype=int32)

In [ ]:
[tuple(a[i]) for i in [0,1,2,0]]

In [ ]:
a.shape

In [ ]:
n_hull= list(scipy.spatial.ConvexHull(a).vertices)
n_hull

# convex_hull = [tuple(verts[i]) for i in n_hull]

In [ ]:
n_hull.append(n_hull[0])

In [ ]:
n_hull

In [ ]:
convex_hull = [tuple(verts[i]) for i in n_hull]
convex_hull

In [ ]:
import cv2

In [ ]:
a = np.array([[[0., 0.  ]],
 [[4.58, 7.14]],
 [[0. ,7.14]],
             ])
a.shape

In [ ]:
hull = cv2.convexHull(a)

In [ ]:
!pip install opencv-python

In [ ]:
src = cv2.imread("p_3.png", 1)

In [ ]:
src.shape

In [ ]:
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY) # convert to grayscale
2
blur = cv2.blur(gray, (3, 3)) # blur the image
3
ret, thresh = cv2.threshold(blur, 50, 255, cv2.THRESH_BINARY)


In [ ]:
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
for i in range(len(contours)):
    print(contours[i].shape)

In [ ]:
contours[0]

In [ ]:
# Python3 program to find convex hull of a set of points. Refer
# https://www.geeksforgeeks.org/orientation-3-ordered-points/
# for explanation of orientation()

# point class with x, y as point
class Point:
	def __init__(self, x, y):
		self.x = x
		self.y = y

def Left_index(points):
	
	'''
	Finding the left most point
	'''
	minn = 0
	for i in range(1,len(points)):
		if points[i].x < points[minn].x:
			minn = i
		elif points[i].x == points[minn].x:
			if points[i].y > points[minn].y:
				minn = i
	return minn

def orientation(p, q, r):
	'''
	To find orientation of ordered triplet (p, q, r).
	The function returns following values
	0 --> p, q and r are collinear
	1 --> Clockwise
	2 --> Counterclockwise
	'''
	val = (q.y - p.y) * (r.x - q.x) - \
		(q.x - p.x) * (r.y - q.y)

	if val == 0:
		return 0
	elif val > 0:
		return 1
	else:
		return 2

def convexHull(points, n):
	
	# There must be at least 3 points
	if n < 3:
		return

	# Find the leftmost point
	l = Left_index(points)

	hull = []
	
	'''
	Start from leftmost point, keep moving counterclockwise
	until reach the start point again. This loop runs O(h)
	times where h is number of points in result or output.
	'''
	p = l
	q = 0
	while(True):
		
		# Add current point to result
		hull.append(p)

		'''
		Search for a point 'q' such that orientation(p, q,
		x) is counterclockwise for all points 'x'. The idea
		is to keep track of last visited most counterclock-
		wise point in q. If any point 'i' is more counterclock-
		wise than q, then update q.
		'''
		q = (p + 1) % n

		for i in range(n):
			
			# If i is more counterclockwise
			# than current q, then update q
			if(orientation(points[p],
						points[i], points[q]) == 2):
				q = i

		'''
		Now q is the most counterclockwise with respect to p
		Set p as q for next iteration, so that q is added to
		result 'hull'
		'''
		p = q

		# While we don't come to first point
		if(p == l):
			break

	# Print Result
	for each in hull:
		print(points[each].x, points[each].y)

# Driver Code

points = []
points.append(Point(0, 0))
points.append(Point(4.58, 7.14))
points.append(Point(0, 7.14))
# points.append(Point(2, 1))
# points.append(Point(3, 0))
# points.append(Point(0, 0))
# points.append(Point(3, 3))

convexHull(points, len(points))

# This code is contributed by
# Akarsh Somani, IIIT Kalyani


In [ ]:
# A Python3 program to find convex hull of a set of points. Refer
# https://www.geeksforgeeks.org/orientation-3-ordered-points/
# for explanation of orientation()

from functools import cmp_to_key

# A class used to store the x and y coordinates of points
class Point:
	def __init__(self, x = None, y = None):
		self.x = x
		self.y = y

# A global point needed for sorting points with reference
# to the first point
p0 = Point(0, 0)

# A utility function to find next to top in a stack
def nextToTop(S):
	return S[-2]

# A utility function to return square of distance
# between p1 and p2
def distSq(p1, p2):
	return ((p1.x - p2.x) * (p1.x - p2.x) +
			(p1.y - p2.y) * (p1.y - p2.y))

# To find orientation of ordered triplet (p, q, r).
# The function returns following values
# 0 --> p, q and r are collinear
# 1 --> Clockwise
# 2 --> Counterclockwise
def orientation(p, q, r):
	val = ((q.y - p.y) * (r.x - q.x) -
		(q.x - p.x) * (r.y - q.y))
	if val == 0:
		return 0 # collinear
	elif val > 0:
		return 1 # clock wise
	else:
		return 2 # counterclock wise

# A function used by cmp_to_key function to sort an array of
# points with respect to the first point
def compare(p1, p2):

	# Find orientation
	o = orientation(p0, p1, p2)
	if o == 0:
		if distSq(p0, p2) >= distSq(p0, p1):
			return -1
		else:
			return 1
	else:
		if o == 2:
			return -1
		else:
			return 1

# Prints convex hull of a set of n points.
def convexHull(points, n):

	# Find the bottommost point
	ymin = points[0].y
	min = 0
	for i in range(1, n):
		y = points[i].y

		# Pick the bottom-most or chose the left
		# most point in case of tie
		if ((y < ymin) or
			(ymin == y and points[i].x < points[min].x)):
			ymin = points[i].y
			min = i

	# Place the bottom-most point at first position
	points[0], points[min] = points[min], points[0]

	# Sort n-1 points with respect to the first point.
	# A point p1 comes before p2 in sorted output if p2
	# has larger polar angle (in counterclockwise
	# direction) than p1
	p0 = points[0]
	points = sorted(points, key=cmp_to_key(compare))

	# If two or more points make same angle with p0,
	# Remove all but the one that is farthest from p0
	# Remember that, in above sorting, our criteria was
	# to keep the farthest point at the end when more than
	# one points have same angle.
	m = 1 # Initialize size of modified array
	for i in range(1, n):
	
		# Keep removing i while angle of i and i+1 is same
		# with respect to p0
		while ((i < n - 1) and
		(orientation(p0, points[i], points[i + 1]) == 0)):
			i += 1

		points[m] = points[i]
		m += 1 # Update size of modified array

	# If modified array of points has less than 3 points,
	# convex hull is not possible
	if m < 3:
		return

	# Create an empty stack and push first three points
	# to it.
	S = []
	S.append(points[0])
	S.append(points[1])
	S.append(points[2])

	# Process remaining n-3 points
	for i in range(3, m):
	
		# Keep removing top while the angle formed by
		# points next-to-top, top, and points[i] makes
		# a non-left turn
		while ((len(S) > 1) and
		(orientation(nextToTop(S), S[-1], points[i]) != 2)):
			S.pop()
		S.append(points[i])

	# Now stack has the output points,
	# print contents of stack
	while S:
		p = S[-1]
		print("(" + str(p.x) + ", " + str(p.y) + ")")
		S.pop()

# Driver Code
input_points = [(0, 0), (4.58, 7.14), (0, 7.14)]
points = []
for point in input_points:
	points.append(Point(point[0], point[1]))
n = len(points)
convexHull(points, n)

# This code is contributed by Kevin Joshi


In [ ]:
a = np.array([[[0., 0.  ]],
 [[4.58, 7.14]],
 [[0. ,7.14]],
             ])
a.shape